
# Sprachmodell-Anfrage mit modernem OpenAI Client und GPT4All-Embeddings

Dieses Jupyter Notebook demonstriert, wie man mit Hilfe des OpenAI-Clients und GPT4All-Embeddings
eine umfassende Textanalyse durchführt. Es verwendet den Hochschulinternen API-Endpunkt.

## Überblick

In diesem Notebook lernen Sie:
1. Wie man die OpenAI-Bibliothek und GPT4All installiert
2. Wie man den modernen OpenAI Client mit benutzerdefinierten Endpunkten konfiguriert
3. Wie man lokale Embeddings mit GPT4All generiert
4. Wie man lange Texte analysiert und mit OpenAI integriert


## 1. Installation der erforderlichen Pakete

Zunächst müssen wir sicherstellen, dass alle notwendigen Bibliotheken installiert sind.

Auf AI.H2.de ist das nicht nötig, da der JupyterHub die Bibliotheken schon vorinstalliert.

In anderen Umgebungen muss das noch nachgeholt werden. Führen Sie die folgende Zelle aus:

In [ ]:
!pip install "openai>=1.0.0" gpt4all langchain langchain_community

## 2. Initialisierung des modernen OpenAI Clients

Wir importieren die `OpenAI`-Klasse und erstellen einen Client mit unseren Konfigurationsparametern.

**Wichtige Parameter:**
- `api_key`: Der Authentifizierungsschlüssel für die API
- `base_url`: Die Basis-URL des API-Servers (ersetzt das frühere `api_base`)

In [ ]:
from openai import OpenAI

# Initialisierung des Clients mit benutzerdefinierten Parametern
client = OpenAI(
    api_key="sk-1234",            # API-Key für die Authentifizierung
    base_url="https://ai.h2.de/llm"  # Benutzerdefinierter Endpunkt
)

print(f"Moderner OpenAI Client initialisiert mit Basis-URL: {client.base_url}")

## 3. Einrichtung des GPT4All-Embeddings-Modells

Wir initialisieren das GPT4All-Embeddings-Modell. Dieses Modell ist besonders geeignet für:
- Lokale Text-Einbettungen
- Effiziente Verarbeitung langer Texte
- Privatsphäre-gerechte Textanalyse

In [ ]:
from gpt4all import Embed4All

# Initialisierung des Embeddings-Modells
embedder = Embed4All()

print("GPT4All-Embeddings-Modell erfolgreich initialisiert")

## 4. Generierung von Embeddings für einen langen Text

In diesem Abschnitt demonstrieren wir, wie man Embeddings für einen langen Text generiert. Das Modell verarbeitet automatisch Texte, die länger als der Kontext sind, indem es sie in Chunks unterteilt und die Embeddings durchschnittlich berechnet.

In [ ]:
# Beispiel für einen langen Text (z.B. ein Artikel oder Dokument)
long_text = """
Die Künstliche Intelligenz (KI) ist eine der wichtigsten Technologien des 21. Jahrhunderts.
Sie revolutioniert Bereiche wie Gesundheitswesen, Finanzwesen, Produktion und Vertrieb.
KI-Systeme können komplexe Muster erkennen, Vorhersagen treffen und selbstständig lernen.
Dabei spielen verschiedene Techniken wie neuronale Netze, maschinelles Lernen und tiefes Lernen
eine zentrale Rolle. Die Entwicklung fortschreitender KI-Modelle wie GPT-4 oder Claude
öffnet neue Möglichkeiten für natürlichsprachliche Interaktionen und intelligente Systeme.
Trotz der Vorteile gibt es auch Herausforderungen wie Datensicherheit, Ethik und
die Notwendigkeit menschlicher Überwachung.
"""

In [ ]:
# Generieren der Embeddings
embeddings = embedder.embed(long_text, long_text_mode="mean")

# Ausgabe der Embedding-Größe
print(f"Generierte Embeddings haben die Dimension: {len(embeddings)}")

## 5. Vorbereitung für kontextbasierte Suche

Wir können Embeddings nutzen, um relevante Passagen aus einem umfangreichen Dokument zu finden und den Kontext für die Anfrage zu erweitern.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Beispiel für ein längeres Dokument (in der Praxis könnten dies mehrere Dokumente sein)
extended_document = """
Die Künstliche Intelligenz (KI) ist eine der wichtigsten Technologien des 21. Jahrhunderts.
Sie revolutioniert Bereiche wie Gesundheitswesen, Finanzwesen, Produktion und Vertrieb.
KI-Systeme können komplexe Muster erkennen, Vorhersagen treffen und selbstständig lernen.

Neuronale Netze sind ein zentrales Konzept in der KI. Sie bestehen aus Schichten von Neuronen,
die miteinander verbunden sind und komplexe Berechnungen durchführen können. Deep Learning,
eine Unterkategorie des maschinellen Lernens, nutzt tiefe neuronale Netze mit vielen
verborgenen Schichten, um besonders komplexe Aufgaben zu bewältigen.

Maschinelles Lernen ermöglicht es Computern, aus Erfahrungen zu lernen und sich anzupassen,
ohne explizit programmiert zu werden. Es gibt verschiedene Ansätze: überwachtes Lernen,
unüberwachtes Lernen und bestärkendes Lernen.

Moderne KI-Modelle wie GPT-4 oder Claude basieren auf Transformer-Architekturen, die
besonders effektiv bei der Verarbeitung natürlicher Sprache sind. Diese Modelle werden
mit enormen Datenmengen trainiert und können beeindruckende Ergebnisse bei Textgenerierung,
Übersetzung und Inhaltsanalyse erzielen.

Die Herausforderungen der KI umfassen ethische Bedenken, Datenschutz, Fairness und Transparenz.
Es ist wichtig, dass KI-Systeme verantwortungsvoll entwickelt und eingesetzt werden.
"""

# Text in Chunks aufteilen für detailliertere Analyse
def split_into_chunks(text, chunk_size=150):
    """Text in überlappende Chunks aufteilen"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size // 2):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

document_chunks = split_into_chunks(extended_document)
print(f"Dokument wurde in {len(document_chunks)} Chunks aufgeteilt")

# Embeddings für jeden Chunk generieren
chunk_embeddings = [embedder.embed(chunk) for chunk in document_chunks]

## 6. Kontexterweiterung für die OpenAI-Anfrage

Mit Hilfe von Embeddings können wir die relevantesten Teile des Dokuments identifizieren
und sie als Kontext in unsere Anfrage an OpenAI einbeziehen.

In [ ]:
# Beispiel-Anfrage
query = "Welche technischen Grundlagen stehen hinter modernen KI-Systemen?"
query_embedding = embedder.embed(query)

# Ähnlichkeiten zu allen Chunks berechnen
similarities = [cosine_similarity([query_embedding], [emb])[0][0] for emb in chunk_embeddings]

# Die relevantesten Chunks auswählen (Top-2)
top_chunk_indices = np.argsort(similarities)[-2:]
relevant_context = "\n".join([document_chunks[i] for i in top_chunk_indices])

print(f"Relevantester Kontext ausgewählt mit Ähnlichkeitswerten: {[similarities[i] for i in top_chunk_indices]}")

# Senden einer Anfrage an das OpenAI-Modell mit erweitertem Kontext
context_enhanced_prompt = f"""
Hier ist relevanter Kontext zu der Anfrage:

{relevant_context}

Basierend auf diesem Kontext, beantworte bitte folgende Frage:
{query}
"""

completion = client.chat.completions.create(
    model="Llama3.3",
    messages=[
        {"role": "system", "content": "Du bist ein hilfreicher Assistent für KI-Themen."},
        {"role": "user", "content": context_enhanced_prompt}
    ],
    temperature=0.3
)

# Formatierte Ausgabe der Antwort
from IPython.display import Markdown, display

markdown_output = f"""
## Antwort mit erweitertem Kontext:

> {completion.choices[0].message.content}

### Metadaten:
- **Modell**: {completion.model}
- **Completion-Tokens**: {completion.usage.completion_tokens}
- **Prompt-Tokens**: {completion.usage.prompt_tokens}
- **Gesamtanzahl der Tokens**: {completion.usage.total_tokens}
"""
display(Markdown(markdown_output))